# Введение в MSTICPy

## Вступление

[MSTICPy](https://msticpy.readthedocs.io/) или `Microsoft Threat Intelligence Python Security Tools` — это набор инструментов на языке Python, предназначенных для расследования инцидентов в области кибербезопасности, поиска индикаторов компрометации (IoC). Многие из инструментов возникли как Jupyter-блокноты, написанные для решения задач форензики. Некоторые инструменты полезны только в блокнотах (например, виджеты и визуализация), но многие другие можно использовать из командной строки или импортировать в свой Python-код.

Пакет отвечает трем основным потребностям в расследовании инцидентов кибербезопасности:

- получение и обогащение данных;
- анализ данных;
- визуализация данных.

### Дополнительно:

Отличная обзорная [статья на Хабре](https://habr.com/ru/company/microsoft/blog/487584/) о том, чем Jupyter-блокноты могут помочь исследователям кибербезопасности.

Также Microsoft ежегодно проводит онлайн конференцию [Infosec Jupyterthon](https://infosecjupyterthon.com/introduction.html) по использованию Jupyter-блокнотов в кибербезопасности.

## Варианты использования и среды

Хотя `MSTICPy` изначально разрабатывался для использования с `Azure Sentinel` (это такая облачная SIEM от Microsoft), большая часть пакета не зависит от источников данных. Также включены компоненты запроса данных для `Splunk` (это платформа для сбора, хранения, обработки и анализа логов), `Microsoft 365 Defender Advanced`, `Microsoft Graph` и других.

По опыту использования `MSTICPy` сильно привязан к облачным API, для которых необходимы лицензии и прочие ключи доступа, что несколько снижает заявленную открытость/доступность/полезность пакета. По сути `MSTICPy` является всего лишь интерфейсом поверх десятка различных API и различных пакетов Python.

API-интерфейсы инструментов обычно используют формат `DataFrame` пакета pandas в качестве входных данных и, при необходимости, возвращают данные в виде `DataFrame`.

## Установка

`MSTICPy` требует Python 3.8 или более поздней версии. У меня получилось установить только с Python 3.11. 

Если вы используете Jupyter-блокноты локально, то потребуется установить Python 3.11. Рекомендую дистрибутив Ananconda, поскольку он поставляется со многими предустановленными пакетами, необходимыми для `MSTICPy`.

`MSTICPy` имеет большое количество зависимостей и, чтобы избежать конфликтов с пакетами в существующей среде Python, вы можете создать виртуальную среду `conda` и установить пакет там.

Для `сonda` используйте команду `conda create` из оболочки `conda`.

```
(base) conda create --name infosec python==3.11
```

Активируем созданное виртуальное окружение:
```
(base) conda activate infosec
```
Следующий шаг - установка `MSTICPy`. Вы можете выбрать несколько конфигураций пакетов, но у меня получилось установить только с полным набором (в MacOS):
```
(infosec) pip install msticpy\[all]
```
PS. или в ОС Windows: 
```
(infosec) pip install msticpy[all]
```
Вручную обновите `MSTICPy` до крайней версии, иначе примеры работать не будут:
```
(infosec) pip install --upgrade msticpy==2.2.0
```
Я предпочитаю использовать оболочку Jupyter Lab, поэтому далее устанавливаю ее:
```
(infosec) conda install -c conda-forge jupyterlab
```
Запукаем Jupyter Lab и радуемся, что все работает:
```
(infosec) jupyter lab
```

Вы можете импортировать `MSTICPy` как есть или переименовать его во что-то более простое для ввода, например `mp`:

In [42]:
import msticpy as mp

Доступна простая помощь:

In [43]:
help(mp)

Help on package msticpy:

NAME
    msticpy - Jupyter and Python Tools for InfoSec.

DESCRIPTION
    -----------------------------------------------
    
    Requires Python 3.8 or later.
    
    To quickly import common modules into a notebook run:
    
    >>> import msticpy as mp
    >>> mp.init_notebook()
    
    If not running in a notebook/IPython use
    >>> mp.init_notebook(globals())
    
    To see help on `init_notebook`:
    >>> help(mp.init_notebook)
    
    Search msticpy modules for a keyword:
    >>> mp.search(keyword)
    
    -----------------------------------------------
    
    Full documentation is available at:
    https://msticpy.readthedocs.io
    
    GitHub repo:
    https://github.com/microsoft/msticpy
    
    
    Package structure:
    
    - analysis - analysis functions
    - auth - authentication and secrets management
    - common - utility functions, common types, exceptions
    - config - configuration tool
    - data - queries, data access, cont

Используйте функцию `search`, чтобы найти необходимый модуль для импорта:

In [44]:
mp.search("geo")

Module,Help
msticpy.context.geoip,msticpy.context.geoip
msticpy.datamodel.entities.geo_location,msticpy.datamodel.entities.geo_location


## Инициализация MSTICpy

Функция инициализации `init_notebook` предназначена для подготовки блокнота. Она делает несколько полезных вещей:

- Импортирует некоторые распространенные (не `MSTICPy`) пакеты, такие как `pandas`, `numpy`, `ipywidgets`.

- Импортирует ряд компонентов `MSTICPy`, таких как `Entities`.

- Проверяет наличие действительного файла конфигурации `msticpyconfig`. Для некоторых элементов `MSTICPy` требуются [параметры конфигурации](https://msticpy.readthedocs.io/en/latest/getting_started/msticpyconfig.html). Примером могут служить поставщики Threat Intelligence, т.е. потоков данных (фидов) с индикаторами компрометации.

- Инициализирует магию блокнота `MSTICPy` и средства доступа к `pandas`.

- Перехватывает обработку исключений блокнота для отображения дружественных исключений `MSTICPy` (другие исключения не затрагиваются).

In [45]:
help(mp.init_notebook)

Help on function init_notebook in module msticpy.init.nbinit:

init_notebook(namespace: Optional[Dict[str, Any]] = None, def_imports: str = 'all', additional_packages: List[str] = None, extra_imports: List[str] = None, **kwargs)
    Initialize the notebook environment.
    
    Parameters
    ----------
    namespace : Dict[str, Any], optional
        Namespace (usually globals()) into which imports
        are to be populated.
        By default, it will use the ipython `user_global_ns`.
    def_imports : str, optional
        Import default packages. By default "all".
        Possible values are:
        - "all" - import all packages
        - "nb" - import common notebook packages
        - "msticpy" - import msticpy packages
        - "none" (or any other value) don't load any default packages.
    additional_packages : List[str], optional
        Additional packages to be pip installed,
        by default None.
        Packages are specified by name only or version
        specifi

In [46]:
mp.init_notebook()

Вы можете заполнить `msticpyconfig` вручную или использовать редактор настроек `MSTICPy` для просмотра и редактирования сохраненных там настроек.

In [47]:
#msticpy.MpConfigEdit()

## Доступ к наборам данных Mordor

Рассмотрим два способо загрузки данных из области кибербезопасности:
- с помощью модуля `requests`;
- с помощью `MSTICPy`.

### Использование requests для доступа к наборам данных Mordor

Проект [Security Datasets](https://securitydatasets.com/introduction.html) — это инициатива с открытым исходным кодом, которая предоставляет предварительно записанные наборы данных, описывающие вредоносные действия с разных платформ, сообществу кибербезопасности для ускорения анализа данных и исследования угроз.

Начнем с импорта необходимых библиотек Python для доступа к содержимому наборов данных:

In [48]:
import requests
from zipfile import ZipFile
from io import BytesIO
from pandas.io import json

Мы сделаем HTTP-запрос к репозиторию [Security Datasets](https://github.com/OTRF/Security-Datasets) с помощью метода `GET` и сохраним содержимое ответа в переменной `zipFileRequest`.

Важно отметить, что мы используем ссылку на необработанные данные, связанную с набором данных. Этот тип ссылок обычно начинается с https://raw.githubusercontent.com/ + ссылка на проект.

In [49]:
url = 'https://raw.githubusercontent.com/OTRF/Security-Datasets/master/datasets/atomic/windows/discovery/host/empire_shell_net_localgroup_administrators.zip'
zipFileRequest = requests.get(url)

In [50]:
type(zipFileRequest)

requests.models.Response

In [51]:
# Тип данных содержимого HTTP-ответа
type(zipFileRequest.content)

bytes

Мы создадим объект [BytesIO](https://docs.python.org/3/library/io.html#io.BytesIO) для доступа к содержимому ответа и сохраним его в объекте [ZipFile](https://docs.python.org/3/library/zipfile.html#zipfile-objects). Все манипуляции с данными выполняются в памяти.

In [52]:
zipFile = ZipFile(BytesIO(zipFileRequest.content))
type(zipFile)

zipfile.ZipFile

Любой объект `ZipFile` может содержать более одного файла. Мы можем получить доступ к списку имен файлов, используя метод [namelist](https://docs.python.org/3/library/zipfile.html#zipfile.ZipFile.namelist). Поскольку наборы данных содержат один файл, то будем ссылаться на первый элемент списка при извлечении файла `JSON`.

In [53]:
zipFile.namelist()

['empire_shell_net_localgroup_administrators_2020-09-21191843.json']

Мы извлечем файл `JSON` из сжатой папки, используя метод [extract](https://docs.python.org/3/library/zipfile.html#zipfile.ZipFile.extract). После запуска приведенного ниже кода загрузим и сохраним файл в каталоге, указанном в параметре `path`.

Важно отметить, что этот метод возвращает нормализованный путь к файлу `JSON`. Мы сохраняем путь к каталогу в переменной `datasetJSONPath` и используем его при попытке прочитать файл.

In [54]:
datasetJSONPath = zipFile.extract(zipFile.namelist()[0], path = '../data')

print(datasetJSONPath)

../data/empire_shell_net_localgroup_administrators_2020-09-21191843.json


Теперь, когда файл загружен и известен путь к нему, мы можем прочитать файл `JSON` с помощью метода [read_json](https://pandas.pydata.org/docs/reference/api/pandas.read_json.html?highlight=read_json#pandas.read_json).

Важно отметить, что при записи набора данных каждая строка файла `JSON` представляет собой событие. Поэтому важно установить для параметра `lines` значение `True`.

In [55]:
dataset = json.read_json(path_or_buf=datasetJSONPath, 
                         lines=True)

Метод `read_json` возвращает объект `DataFrame`:

In [56]:
type(dataset)

pandas.core.frame.DataFrame

Наконец, мы должны начать исследовать наш набор данных, используя различные функции или методы, такие как `head`.

In [57]:
dataset.head(n=1)

,Keywords,SeverityValue,TargetObject,EventTypeOrignal,EventID,ProviderGuid,ExecutionProcessID,host,Channel,UserID,AccountType,ThreadID,ProcessGuid,EventReceivedTime,Opcode,EventTime,@timestamp,SourceModuleType,port,AccountName,RecordNumber,Task,Domain,@version,OpcodeValue,...,AccessReason,AccessList,RestrictedSidCount,ResourceAttributes,CallerProcessName,TargetSid,CallerProcessId,Status,SourcePortName,DestinationPort,SourceHostname,DestinationIp,SourceIp,DestinationIsIpv6,Initiated,SourceIsIpv6,DestinationPortName,DestinationHostname,Service,Details,ShareName,EnabledPrivilegeList,DisabledPrivilegeList,ShareLocalPath,RelativeTargetName
0,-9223372036854775808,2,HKU\S-1-5-21-4228717743-1032521047-1810997296-1104\Software\Microsoft\Windows\CurrentVersion\Int...,INFO,12,{5770385F-C22A-43E0-BF4C-06F5698FFBD9},3172,wec.internal.cloudapp.net,Microsoft-Windows-Sysmon/Operational,S-1-5-18,User,4048,{b34bc01c-2f95-5f69-5f01-000000000900},2020-09-21 19:18:44,Info,2020-09-21 19:18:41,2020-09-21T23:18:44.265Z,im_msvistalog,64545,SYSTEM,3226968,12,NT AUTHORITY,1,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Использование MSTICPy для доступа к наборам данных Mordor

In [58]:
import pandas as pd
from msticpy.data import QueryProvider
from msticpy.vis import mp_pandas_plot

Чтобы использовать [Mordor провайдер](https://msticpy.readthedocs.io/en/latest/data_acquisition/MordorData.html), сначала создайте провайдер запросов `Mordor`. Затем вызовите функцию `connect`: она загрузит метаданные из `Mordor` и `Mitre` для заполнения набора запросов.

In [59]:
qry_prov_sd = QueryProvider("Mordor")

Ход загрузки отображается с помощью индикатора выполнения.

In [60]:
qry_prov_sd.connect()

Retrieving Mitre data...
Retrieving Mordor data...


После загрузки метаданных поставщик заполняется функциями запроса, которые можно использовать для извлечения наборов данных.

Вы можете увидеть список доступных запросов с помощью функции `list_queries`.

In [61]:
qry_prov_sd.list_queries()[:10]

['atomic.aws.collection.ec2_proxy_s3_exfiltration',
 'atomic.aws.discovery.aws_s3_honeybucketlogs',
 'atomic.linux.defense_evasion.host.sh_binary_padding_dd',
 'atomic.linux.discovery.host.sh_arp_cache',
 'atomic.windows.collection.host.msf_record_mic',
 'atomic.windows.credential_access.host.cmd_lsass_memory_dumpert_syscalls',
 'atomic.windows.credential_access.host.cmd_psexec_lsa_secrets_dump',
 'atomic.windows.credential_access.host.cmd_sam_copy_esentutl',
 'atomic.windows.credential_access.host.covenant_dcsync_dcerpc_drsuapi_DsGetNCChanges',
 'atomic.windows.credential_access.host.empire_dcsync_dcerpc_drsuapi_DsGetNCChanges']

Вы можете использовать функцию провайдера `search_queries` для поиска запросов на соответствие требуемым атрибутам.

In [62]:
qry_prov_sd.search_queries("empire + localgroup")

['atomic.windows.discovery.host.empire_shell_net_localgroup_administrators (Empire Net Local Administrators Group)']

Чтобы получить набор данных, выполните требуемый запрос. Все запросы доступны как атрибуты провайдера `Mordor`.

In [63]:
emp_df = qry_prov_sd.atomic.windows.discovery.host.empire_shell_net_localgroup_administrators()
emp_df.head()

https://raw.githubusercontent.com/OTRF/Security-Datasets/master/datasets/atomic/windows/discovery/host/empire_shell_net_localgroup_administrators.zip
Extracting empire_shell_net_localgroup_administrators_2020-09-21191843.json


,Keywords,SeverityValue,TargetObject,EventTypeOrignal,EventID,ProviderGuid,ExecutionProcessID,host,Channel,UserID,AccountType,ThreadID,ProcessGuid,EventReceivedTime,Opcode,EventTime,@timestamp,SourceModuleType,port,AccountName,RecordNumber,Task,Domain,@version,OpcodeValue,...,AccessReason,AccessList,RestrictedSidCount,ResourceAttributes,CallerProcessName,TargetSid,CallerProcessId,Status,SourcePortName,DestinationPort,SourceHostname,DestinationIp,SourceIp,DestinationIsIpv6,Initiated,SourceIsIpv6,DestinationPortName,DestinationHostname,Service,Details,ShareName,EnabledPrivilegeList,DisabledPrivilegeList,ShareLocalPath,RelativeTargetName
0,-9223372036854775808,2,HKU\S-1-5-21-4228717743-1032521047-1810997296-1104\Software\Microsoft\Windows\CurrentVersion\Int...,INFO,12,{5770385F-C22A-43E0-BF4C-06F5698FFBD9},3172,wec.internal.cloudapp.net,Microsoft-Windows-Sysmon/Operational,S-1-5-18,User,4048,{b34bc01c-2f95-5f69-5f01-000000000900},2020-09-21 19:18:44,Info,2020-09-21 19:18:41,2020-09-21T23:18:44.265Z,im_msvistalog,64545,SYSTEM,3226968,12,NT AUTHORITY,1,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,2,NaN,NaN,4103,{A0C1853B-5C40-4B15-8766-3CF1C58F985A},7456,wec.internal.cloudapp.net,Microsoft-Windows-PowerShell/Operational,S-1-5-21-4228717743-1032521047-1810997296-1104,User,840,NaN,2020-09-21 19:18:44,To be used when operation is just executing a method,2020-09-21 19:18:41,2020-09-21T23:18:44.265Z,im_msvistalog,64545,pgustavo,162586,106,THESHIRE,1,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,2,NaN,NaN,4103,{A0C1853B-5C40-4B15-8766-3CF1C58F985A},7456,wec.internal.cloudapp.net,Microsoft-Windows-PowerShell/Operational,S-1-5-21-4228717743-1032521047-1810997296-1104,User,840,NaN,2020-09-21 19:18:44,To be used when operation is just executing a method,2020-09-21 19:18:41,2020-09-21T23:18:44.266Z,im_msvistalog,64545,pgustavo,162587,106,THESHIRE,1,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-9214364837600034816,2,NaN,NaN,5158,{54849625-5478-4994-A5BA-3E3B0328C30D},4,wec.internal.cloudapp.net,Security,NaN,NaN,1536,NaN,2020-09-21 19:18:44,Info,2020-09-21 19:18:41,2020-09-21T23:18:44.267Z,im_msvistalog,64545,NaN,1901286,12810,NaN,1,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-9214364837600034816,2,NaN,NaN,5156,{54849625-5478-4994-A5BA-3E3B0328C30D},4,wec.internal.cloudapp.net,Security,NaN,NaN,1536,NaN,2020-09-21 19:18:44,Info,2020-09-21 19:18:41,2020-09-21T23:18:44.267Z,im_msvistalog,64545,NaN,1901287,12810,NaN,1,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Убедитесь, что временные метки действительно являются временными метками, а не строками.

In [64]:
emp_df["EventTime"] = pd.to_datetime(emp_df["EventTime"])

In [65]:
emp_df.mp_plot.timeline(time_column="EventTime", 
                        group_by="EventID")

Loading BokehJS ...

Column(id='2577', ...)

## Виджеты MSTICPy

`MSTICPy` включает ряд виджетов, упрощающих взаимодействие с данными, особенно для пользователей, не имеющих опыта программирования.

Виджеты предназначены для выполнения ряда общих задач, которые могут потребоваться пользователю для взаимодействия с блокнотом, таких как выбор элементов из возвращенных данных или установка временных рамок для запроса.

Сами виджеты встроены в `ipywidgets` и доступны в модуле `msticpy.nbtools.nbwidgets`.

    Примечание. Виджеты автоматически импортируются программой init_notebook.

Приведенный ниже код создает виджет `Временной диапазон`, который можно использовать, чтобы позволить пользователю установить временной диапазон. Мы говорим ему использовать дни в качестве единицы измерения и устанавливаем максимальный диапазон для выбора.

In [66]:
from msticpy.nbtools.nbwidgets import *

time_select = QueryTime(units="day", 
                        max_before=20, 
                        before=5, 
                        max_after=1)
time_select.display()

Затем мы можем вызвать свойства `start` / `end` и получить объекты даты и времени на основе выбора пользователя.

In [67]:
time_select.start

datetime.datetime(2023, 1, 2, 11, 0, 31, 313226)

Другие виджеты позволяют выбирать элементы из списка вместе с опцией текстового фильтра, чтобы помочь пользователям найти элементы:

In [68]:
items = ["item 1", "item 2", "item 3"]

selection = SelectItem(item_list=items, 
                       description="Select item", 
                       auto_display=True)

Существуют также специальные виджеты, такие как `SelectAlert`, которые позволяют пользователю выбрать конкретное предупреждение из списка предупреждений. 

In [69]:
import pandas as pd
from msticpy.nbtools.nbdisplay import display_alert

alerts = pd.read_pickle("https://github.com/microsoft/msticpy/raw/main/tests/testdata/localdata/alerts_list.pkl")

alert_select = SelectAlert(alerts=alerts, 
                           action=display_alert)
alert_select.display()

,21
TenantId,52b1ab41-869e-4138-9e40-2a4457f09bf0
TimeGenerated,2019-02-15 03:51:09
AlertDisplayName,Detected suspicious file download
AlertName,Detected suspicious file download
Severity,Low
Description,Analysis of host data has detected suspicious download of remote file on MSTICALERTSLXVM2.
ProviderName,Detection
VendorName,Microsoft
VendorOriginalId,caab1270-55d3-4447-8618-16cf8672e4e1
SystemAlertId,2518520981440769999_caab1270-55d3-4447-8618-16cf8672e4e1


,0
TenantId,52b1ab41-869e-4138-9e40-2a4457f09bf0
TimeGenerated,2019-02-18 02:29:07
AlertDisplayName,SSH Anomalous Login ML
AlertName,SSH Anomalous Login ML
Severity,Low
Description,Anomalous login detected for SSH account
ProviderName,CustomAlertRule
VendorName,Alert Rule
VendorOriginalId,b0e143b8-4fa8-47bc-8bc1-9780c8b75541
SystemAlertId,f1ce87ca-8863-4a66-a0bd-a4d3776a7c64


Другие виджеты `MSTICPy` включают:

- Простой слайдер обратного просмотра на основе даты и времени `Lookback`

- Текстовое поле для захвата пользовательского ввода `GetText`

- Виджет для захвата и возврата переменной среды `GetEnvrionmentKey`

- Виджет для выбора подмножества элементов из списка `SelectSubset`

- Виджет, показывающий ход выполнения задачи `Progress`

- Кнопки с несколькими вариантами с функцией ожидания, которая приостанавливает выполнение ячейки до тех пор, пока пользователь не выберет вариант `OptionButtons`

- Более подробную информацию о виджетах `MSTICPy` можно найти [здесь](https://msticpy.readthedocs.io/en/latest/visualization/NotebookWidgets.html).

Примеры официальных блокнотов по [ссылке](https://msticpy.readthedocs.io/en/latest/notebooksamples.html).